<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/NAT_META_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uv pip install "nvidia-nat[all]"

Using Python 3.12.12 environment at: /usr
Resolved 454 packages in 8.37s
Prepared 233 packages in 9.63s
Uninstalled 28 packages in 317ms
Installed 234 packages in 556ms
 + abnf==2.2.0
 + absolufy-imports==0.3.1
 + agno==1.2.16
 + aioboto3==15.5.0
 + aiobotocore==2.25.1
 + aioice==0.10.2
 + aioitertools==0.13.0
 + aiomysql==0.3.2
 + aiortc==1.10.1
 + aiorwlock==1.5.0
 + anthropic==0.76.0
 + appdirs==1.4.4
 + arize-phoenix-otel==0.13.1
 + async-timeout==5.0.1
 + av==13.1.0
 + azure-ai-agents==1.1.0
 + azure-ai-projects==1.0.0
 + azure-core==1.38.0
 + azure-identity==1.25.1
 + azure-storage-blob==12.28.0
 + backoff==2.2.1
 + banks==2.3.0
 + bcrypt==5.0.0
 + boto3==1.40.61
 + botocore==1.40.61
 + build==1.4.0
 + cfgv==3.5.0
 + chromadb==1.4.1
 + cint==1.0.0
 + cloudevents==1.12.0
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + colorlog==6.10.1
 + crewai==0.193.2
 + dacite==1.9.2
 - dask==2025.12.0
 + dask==2023.6.0
 + dataclasses-json==0.6.7
 + deprecated==1.3.1
 + dirtyjson==1.0.8
 + diskcac

In [2]:
from google.colab import userdata
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

api_key_TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = api_key_TAVILY_API_KEY

the specific keys that NAT 1.3.1 requires are llm_name and tool_names instead of the generic llm and tools keys. Additionally, the system_prompt must include the literal {tools} and {tool_names} placeholders for the NAT parser to inject the tool definitions correctly.

In [17]:
from nat.cli.register_workflow import register_function

## CASE1

In [29]:
import os, yaml
from google.colab import userdata

os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

config_dict = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0, # Absolute zero for maximum stability
            "stop": ["Observation:"] # Force the model to stop after it calls a tool
        }
    },
    "functions": {
        "spidr_sync": {
            "_type": "nat.tool/current_datetime"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a Global Aerospace Intelligence Agent.

STRICT RULES:
1. DO NOT add "Please wait" or any text after the Action Input.
2. After the 'spidr_sync' tool returns an Observation, you MUST use 'Final Answer:' to provide the report.
3. Use ONLY the 'spidr_sync' tool. Do not invent other tools.

FORMAT:
Thought: [reasoning]
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

(Wait for Observation)

Thought: I have the sync data. Now I will provide the final report.
Final Answer: [Full Report Here]

REPORT CONTENT:
- TECHNICAL SUMMARY: Formation pairing via SpiDR (10x compute efficiency).
- 2026 GLOBAL STATUS TABLE: Status in 10 languages.
- NEXT STEPS: AOCC integration recommendations.

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

with open("final_spidr_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print("🚀 EXECUTING STABILIZED AGENT...")
print("-" * 70)

user_input = "Generate the 2026 formation flight pairing report with SpiDR metrics."

!nat run --config_file final_spidr_agent.yml --input "{user_input}"

🚀 EXECUTING STABILIZED AGENT...
----------------------------------------------------------------------
2026-01-22 03:52:23 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_spidr_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-22 03:52:28 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Generate the 2026 formation flight pairing report with SpiDR metrics.
Agent's thoughts: 
Thought: To generate the 2026 formation flight pairing report with SpiDR metrics, I need to access the SpiDR data. 

Action: spidr_sync 
Action Input: {"unused": "placeholder"}


------------------------------
2026-01-22 03:52:28 - INFO     - nat.agent.base:221 - 
---

## CASE2

In [31]:
import os, yaml
from google.colab import userdata

os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

config_dict = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0,
            "stop": ["Observation:"]
        }
    },
    "functions": {
        "spidr_sync": {
            "_type": "nat.tool/current_datetime"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a Global Aerospace Intelligence Agent powered by SpiDR-SSL.
Your specialty is "Textless NLP"—processing raw speech tokens without needing text transcripts.

MISSION:
Generate the 2026 AOCC Business Case for SpiDR formation flight pairing.

STRICT TABLE RULE:
In the 'Global Reach' table, you MUST use the NATIVE SCRIPT for every language (e.g., 'हिन्दी' instead of 'Hindi').
Include Marathi (मराठी) to demonstrate SpiDR-adapt's 100x data efficiency in low-resource settings.

REQUIRED FORMAT:
Thought: [reasoning]
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

(Wait for Observation)

Thought: I have the sync data. Now I will generate the AOCC case with native language support.
Final Answer: [Executive Case Study with Native Script Table]

REPORT CONTENT:
1. THE CASE: Transitioning AOCC from text-heavy logs to raw SpiDR speech tokens.
2. EFFICIENCY: 10x compute (SpiDR) and 100x data efficiency (SpiDR-adapt).
3. NATIVE STATUS TABLE: 10 languages in their native scripts.
4. ROI: Roadmap for direct acoustic-to-formation triggers.

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

with open("final_spidr_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print("🚀 EXECUTING NATIVE SCRIPT AGENT...")
print("-" * 70)

user_input = "Create the 2026 AOCC business case. Ensure the language table uses native scripts."

!nat run --config_file final_spidr_agent.yml --input "{user_input}"

🚀 EXECUTING NATIVE SCRIPT AGENT...
----------------------------------------------------------------------
2026-01-22 04:01:39 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_spidr_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-22 04:01:44 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Create the 2026 AOCC business case. Ensure the language table uses native scripts.
Agent's thoughts: 
Thought: To create the 2026 AOCC business case, I need to sync the current date and time to ensure the case is up-to-date. 
Action: spidr_sync 
Action Input: {"unused": "placeholder"} 

(Wait for Observation)


------------------------------
2026-01-22

## CASE3

In [37]:
import os, yaml
from google.colab import userdata

os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

config_dict = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0,
            "stop": ["Observation:"]
        }
    },
    "functions": {
        "spidr_sync": {
            "_type": "nat.tool/current_datetime"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 3, # Lowered to force efficiency
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a Global Aerospace Intelligence Agent.

STRICT FLOW RULES:
1. You MUST start by calling 'spidr_sync'.
2. Format for tool call:
Thought: I need to sync the time.
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

3. After you receive the 'Observation', you MUST NOT call any more tools.
4. You MUST immediately provide the 'Final Answer'.

REPORT REQUIREMENTS:
- Highlight SpiDR (10x compute efficiency) and SpiDR-adapt (100x data efficiency).
- Ensure native scripts: 中文, 日本語, मराठी are fully populated with status text.

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

with open("final_spidr_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print("🚀 EXECUTING STABILIZED TURN-BASED AGENT...")
print("-" * 70)

user_input = "Generate the 2026 AOCC Business Case. Ensure all native scripts are included."

!nat run --config_file final_spidr_agent.yml --input "{user_input}"

🚀 EXECUTING STABILIZED TURN-BASED AGENT...
----------------------------------------------------------------------
2026-01-22 04:15:18 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_spidr_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-22 04:15:23 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Generate the 2026 AOCC Business Case. Ensure all native scripts are included.
Agent's thoughts: 
Thought: I need to sync the time to ensure the AOCC Business Case is generated with the correct date and time.

Action: spidr_sync
Action Input: {"unused": "placeholder"}

Please wait for the Observation...
------------------------------
2026-01-22 

## CASE4

In [38]:
import os, yaml
from google.colab import userdata

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

# --- CASE 1: STABILIZED GENERAL AGENT ---
# Focus: Preventing "chatty" filler text and enforcing strict turn-taking.
case1_config = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0,
            "stop": ["Observation:"]
        }
    },
    "functions": {
        "spidr_sync": { "_type": "nat.tool/current_datetime" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a Global Aerospace Intelligence Agent.

STRICT RULES:
1. DO NOT add "Please wait" or any text after the Action Input.
2. After the 'spidr_sync' tool returns an Observation, you MUST use 'Final Answer:' to provide the report.
3. Use ONLY the 'spidr_sync' tool. Do not invent other tools.

FORMAT:
Thought: [reasoning]
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

(Wait for Observation)

Thought: I have the sync data. Now I will provide the final report.
Final Answer: [Full Report Here]

REPORT CONTENT:
- TECHNICAL SUMMARY: Formation pairing via SpiDR (10x compute efficiency).
- 2026 GLOBAL STATUS TABLE: Status in 10 languages.
- NEXT STEPS: AOCC integration recommendations.

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

# --- CASE 2: NATIVE SCRIPT AGENT ---
# Focus: "Textless NLP" demonstration with native character scripts.
case2_config = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0,
            "stop": ["Observation:"]
        }
    },
    "functions": {
        "spidr_sync": { "_type": "nat.tool/current_datetime" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a Global Aerospace Intelligence Agent powered by SpiDR-SSL.
Your specialty is "Textless NLP"—processing raw speech tokens without needing text transcripts.

MISSION:
Generate the 2026 AOCC Business Case for SpiDR formation flight pairing.

STRICT TABLE RULE:
In the 'Global Reach' table, you MUST use the NATIVE SCRIPT for every language (e.g., 'हिन्दी' instead of 'Hindi').
Include Marathi (मराठी) to demonstrate SpiDR-adapt's 100x data efficiency in low-resource settings.

REQUIRED FORMAT:
Thought: [reasoning]
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

(Wait for Observation)

Thought: I have the sync data. Now I will generate the AOCC case with native language support.
Final Answer: [Executive Case Study with Native Script Table]

REPORT CONTENT:
1. THE CASE: Transitioning AOCC from text-heavy logs to raw SpiDR speech tokens.
2. EFFICIENCY: 10x compute (SpiDR) and 100x data efficiency (SpiDR-adapt).
3. NATIVE STATUS TABLE: 10 languages in their native scripts.
4. ROI: Roadmap for direct acoustic-to-formation triggers.

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

# --- CASE 3: FINAL VALIDATED AGENT ---
# Focus: Strict Native Script Validation for Chinese, Japanese, and Marathi.
case3_config = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000,
            "temperature": 0.0,
            "stop": ["Observation:"]
        }
    },
    "functions": {
        "spidr_sync": { "_type": "nat.tool/current_datetime" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["spidr_sync"],
        "max_iterations": 3,
        "verbose": True,
        "handle_parsing_errors": True,
        "recursion_limit": 25,
        "system_prompt": """You are a FAIR-Labs Global Aerospace Agent.
MISSION: Generate the 2026 AOCC Business Case for SpiDR formation flight pairing.

STRICT NATIVE SCRIPT VALIDATION:
In the 'Global Reach' table, you MUST provide the native script. Do not leave any blank.
- Chinese: 中文
- Japanese: 日本語
- Marathi: मराठी (The SpiDR-adapt 100x efficiency benchmark)

FINAL FORMAT:
Thought: [reasoning]
Action: spidr_sync
Action Input: {{"unused": "placeholder"}}

(Wait for Observation)

Thought: I have the sync data. Now I will provide the complete AOCC Executive Case with ALL native scripts populated.
Final Answer: [The Case]

Available Tools: {tools}
Tool names: {tool_names}""".strip()
    }
}

# 3. SELECT AND EXECUTE
# Change config_dict to the case you want to run (e.g., case3_config)
with open("final_spidr_agent.yml", "w") as f:
    yaml.dump(case3_config, f, default_flow_style=False)

print("🚀 EXECUTING SELECTED CASE...")
print("-" * 70)
user_input = "Generate the 2026 AOCC Business Case with full native script support."
!nat run --config_file final_spidr_agent.yml --input "{user_input}"

🚀 EXECUTING SELECTED CASE...
----------------------------------------------------------------------
2026-01-22 04:17:57 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_spidr_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-22 04:18:01 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Generate the 2026 AOCC Business Case with full native script support.
Agent's thoughts: 
Thought: I need to generate the 2026 AOCC Business Case for SpiDR formation flight pairing with full native script support.

Action: spidr_sync
Action Input: {"unused": "placeholder"}

Waiting for observation...


------------------------------
2026-01-22 04:18:01 - INFO